Name: **Yuvraj Deshmukh**<br>
Div: **BE11-P11**<br>
Roll no: **43309**<br>
Title: Assignment 6: Object detection using Transfer Learning of CNN architectures <br>

In [1]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import torchvision
from torchvision import models
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import DataLoader  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

In [4]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_classes = 10
learning_rate = 1e-3
batch_size = 1024
num_epochs = 10

In [5]:
# Simple Identity class that let's input pass without changes
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

### 1. Load in a pretrained model (VGG16)

In [6]:
model = models.vgg16(pretrained = True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [7]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### 2. Freezing parameters in model's lower layers

In [8]:
# If you want to do finetuning then set requires_grad = False
for param in model.parameters():
    param.requires_grad = False

In [9]:
# Freezing the average pool layer of the model
model.avgpool = Identity()

### 3. Add custom classifier with several layers of trainable parameters to mode

In [10]:
model.classifier = nn.Sequential(
    nn.Linear(512, 100), nn.ReLU(), 
    nn.Linear(100, num_classes)
)
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### 3. Load Dataset

In [11]:
train_dataset = datasets.CIFAR10(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting dataset/cifar-10-python.tar.gz to dataset/


In [12]:
# Reduce size due to time constraints
newdata = []
for i in range(100):
    newdata.append(train_dataset[i])
newdata   
    

[(tensor([[[0.2314, 0.1686, 0.1961,  ..., 0.6196, 0.5961, 0.5804],
           [0.0627, 0.0000, 0.0706,  ..., 0.4824, 0.4667, 0.4784],
           [0.0980, 0.0627, 0.1922,  ..., 0.4627, 0.4706, 0.4275],
           ...,
           [0.8157, 0.7882, 0.7765,  ..., 0.6275, 0.2196, 0.2078],
           [0.7059, 0.6784, 0.7294,  ..., 0.7216, 0.3804, 0.3255],
           [0.6941, 0.6588, 0.7020,  ..., 0.8471, 0.5922, 0.4824]],
  
          [[0.2431, 0.1804, 0.1882,  ..., 0.5176, 0.4902, 0.4863],
           [0.0784, 0.0000, 0.0314,  ..., 0.3451, 0.3255, 0.3412],
           [0.0941, 0.0275, 0.1059,  ..., 0.3294, 0.3294, 0.2863],
           ...,
           [0.6667, 0.6000, 0.6314,  ..., 0.5216, 0.1216, 0.1333],
           [0.5451, 0.4824, 0.5647,  ..., 0.5804, 0.2431, 0.2078],
           [0.5647, 0.5059, 0.5569,  ..., 0.7216, 0.4627, 0.3608]],
  
          [[0.2471, 0.1765, 0.1686,  ..., 0.4235, 0.4000, 0.4039],
           [0.0784, 0.0000, 0.0000,  ..., 0.2157, 0.1961, 0.2235],
           [0.0824, 0.

In [13]:
train_loader = DataLoader(dataset=newdata, batch_size=batch_size, shuffle=True)

### 4. Train Classifiers on Training Data

In [14]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")

Cost at epoch 0 is 2.28889
Cost at epoch 1 is 2.18888
Cost at epoch 2 is 2.09729
Cost at epoch 3 is 2.01113
Cost at epoch 4 is 1.92871
Cost at epoch 5 is 1.84923
Cost at epoch 6 is 1.77164
Cost at epoch 7 is 1.69543
Cost at epoch 8 is 1.62038
Cost at epoch 9 is 1.54715


### 5. Check Accuracy

In [16]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

In [17]:
check_accuracy(train_loader, model)

Got 73 / 100 with accuracy 73.00
